In [1]:
from tool import *
import plotly.graph_objects as go

In [2]:
with open('D:/Data/IDC_Monitor.bin', 'rb') as f:
    idc = pickle.load(f)

In [3]:
idc

,Curved,Aspect Ratio,Product Detail,OS,Resolution Name,Screen Resolution,Panel Type,Screen Size Detail,Screen Ratio,Product,...,Company,Country,Region,Year,Quarter,Units,Interface,Product Group,RJ45,Value (US$M)
0,No,16:10,TV Monitor,NaN,WXGA+,1440x900,TN,"19"" W",Widescreen,LED,...,Others,United Arab Emirates,Middle East & Africa,2018,2018Q2,2.0,Regular,PCM,NaN,0.000302
1,No,16:10,TV Monitor,NaN,WXGA+,1440x900,TN,"19"" W",Widescreen,LED,...,Others,Saudi Arabia,Middle East & Africa,2018,2018Q2,22.0,Regular,PCM,NaN,0.003366
2,No,16:10,TV Monitor,NaN,WXGA+,1440x900,TN,"19"" W",Widescreen,LED,...,Others,Rest of Africa,Middle East & Africa,2018,2018Q2,33.0,Regular,PCM,NaN,0.005214
3,No,16:10,TV Monitor,NaN,WXGA+,1440x900,TN,"19"" W",Widescreen,LED,...,Others,Jordan,Middle East & Africa,2018,2018Q2,6.0,Regular,PCM,NaN,0.000924
4,No,16:10,TV Monitor,NaN,WXGA+,1440x900,TN,"19"" W",Widescreen,LED,...,Others,Israel,Middle East & Africa,2018,2018Q2,22.0,Regular,PCM,NaN,0.003366
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
780729,No,16:9,Tuneless,NaN,FHD,1920x1080,TN,"21.5"" W",Widescreen,LED,...,Lenovo,Netherlands,Western Europe,2015,2015Q3,1220.0,Regular,PCM,NaN,0.173078
780730,No,16:9,Tuneless,NaN,FHD,1920x1080,VA,"24"" W",Widescreen,LED,...,Acer Group,Thailand,APeJC,2015,2015Q3,1.0,Regular,PCM,NaN,0.000227
780731,No,5:4,Tuneless,NaN,SXGA,1280x1024,IPS,"19"" S",Standard,LED,...,HP Inc,Slovenia,Central & Eastern Europe,2015,2015Q3,93.0,Regular,PCM,NaN,0.016089
780732,No,16:9,Tuneless,NaN,WQHD,2560x1440,IPS,"27"" W",Widescreen,LED,...,Dell Technologies,Romania,Central & Eastern Europe,2015,2015Q3,18.0,Regular,PCM,NaN,0.009057


In [5]:
idc1 = pd.read_excel(get_filename(), sheet_name='Raw')

In [10]:
with open('D:/Data/IDC Monitor_2023_Q4.bin', 'wb') as f:
    pickle.dump(idc1, f)

In [11]:
go.Figure(data=go.Bar(x=idc.groupby('Year').sum('Units')['Units'].index, y =idc.groupby('Year').sum('Units')['Units']))

In [12]:
idc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 780734 entries, 0 to 780733
Data columns (total 24 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Curved              780734 non-null  object 
 1   Aspect Ratio        780734 non-null  object 
 2   Product Detail      780734 non-null  object 
 3   OS                  4273 non-null    object 
 4   Resolution Name     780734 non-null  object 
 5   Screen Resolution   780734 non-null  object 
 6   Panel Type          776047 non-null  object 
 7   Screen Size Detail  780734 non-null  object 
 8   Screen Ratio        780734 non-null  object 
 9   Product             780734 non-null  object 
 10  Product Category    780734 non-null  object 
 11  USB-C               371467 non-null  object 
 12  Brand               780734 non-null  object 
 13  Vendor              780734 non-null  object 
 14  Company             780734 non-null  object 
 15  Country             780734 non-nul

In [40]:
year = 2023
category = 'Aspect Ratio'
# 연도별 category에 해당하는 수량 분포를 bar plot으로 시각화
go.Figure(data=go.Bar(x=idc.groupby('Year').get_group(year).groupby(category).sum('Units')['Units'].index, y=idc.groupby('Year').get_group(year).groupby(category).sum('Units')['Units']))

In [7]:
with open('D:/Data/IDC_Monitor.bin', 'wb') as f:
    pickle.dump(idc, f)

# 업체별 선적 실적

In [13]:
with open('D:/Data/ODM_SR.db', 'rb') as f:
    sr = pickle.load(f)

In [14]:
sr = sr.reset_index()
sr['Week Name'] = sr['Ship Date'].map(get_weekname)
sr['To Site'] = sr['To Site'].replace(ship_to_map)
sr = pd.merge(sr, site[['Region', 'Subsidiary', 'To Site', 'Country']], how='left')
sr[['Year', 'Month']] = sr['Week Name'].apply(lambda x:f'{datetime.date.fromisoformat(x[:10]).isocalendar().year}-{get_month_from_date(datetime.date.fromisoformat(x[:10])):02d}').str.split('-', expand=True)

In [21]:
vendor = 'WINGTECH'
sr = sr[sr['From Site'].str.contains(vendor)].reset_index(drop=True)
c1 = sr['Mapping Model.Suffix'].str.match('\d{2}[T|Z|U|A][G|D|N|W|C]?\d{2}[A-Z]{1,2}') # PC/태블릿 모델일 경우
sr.loc[c1, 'Series'] = sr[c1]['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).str.replace('D|G|N|W|C', '').apply(lambda x:x[:6]) # PC모델의 대표모델명 추출
sr

C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_5776\3749900858.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  sr.loc[c1, 'Series'] = sr[c1]['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).str.replace('D|G|N|W|C', '').apply(lambda x:x[:6]) # PC모델의 대표모델명 추출


,From Site,PO No.,BL No,Mapping Model.Suffix,To Site,Shipping,Cancel,PO,OQC Report,OQC Date,...,SO No.,Final Destination,Division,Week Name,Region,Subsidiary,Country,Year,Month,Series
0,OS_WINGTECH_CN_P,ZPCZ23004308,BAA23070155,10A30Q-L.AQ12ML,EKHQ,Y,N,3040.0,No Report,NaN,...,NaN,NaN,PC,2023-07-17(W29),KOREA,LGEKR,Korea,2023,07,10A30Q
1,OS_WINGTECH_CN_P,ZPCZ23004308,BAA23080105,10A30Q-L.AQ12ML,EKHQ,Y,N,1440.0,No Report,NaN,...,NaN,NaN,PC,2023-08-07(W32),KOREA,LGEKR,Korea,2023,08,10A30Q
2,OS_WINGTECH_CN_P,ZPCZ23004308,BAA23080106,10A30Q-L.AQ12ML,EKHQ,Y,N,160.0,No Report,NaN,...,NaN,NaN,PC,2023-08-14(W33),KOREA,LGEKR,Korea,2023,08,10A30Q
3,OS_WINGTECH_CN_P,ZPCZ23004308,BAA23080384,10A30Q-L.AQ12ML,EKHQ,Y,N,70.0,No Report,NaN,...,NaN,NaN,PC,2023-09-04(W36),KOREA,LGEKR,Korea,2023,09,10A30Q
4,OS_WINGTECH_CN_P,ZPCZ23005449,BAA23090071,10A30Q-L.AQ12ML,EKHQ,Y,N,30.0,No Report,NaN,...,NaN,NaN,PC,2023-09-04(W36),KOREA,LGEKR,Korea,2023,09,10A30Q
5,OS_WINGTECH_CN_P,ZPCZ23004071,BAA23080102,10A30Q-L.AQ14K,EKHQ,Y,N,440.0,No Report,NaN,...,NaN,NaN,PC,2023-08-07(W32),KOREA,LGEKR,Korea,2023,08,10A30Q
6,OS_WINGTECH_CN_P,ZPCZ23004574,BAA23090072,10A30Q-L.AQ14K,EKHQ,Y,N,200.0,No Report,NaN,...,NaN,NaN,PC,2023-09-04(W36),KOREA,LGEKR,Korea,2023,09,10A30Q
7,OS_WINGTECH_CN_P,ZPCZ23004578,BAA23080385,10A30Q-L.AQ1TL,EKHQ,Y,N,440.0,No Report,NaN,...,NaN,NaN,PC,2023-08-28(W35),KOREA,LGEKR,Korea,2023,08,10A30Q
8,OS_WINGTECH_CN_P,ZPCZ23004578,BAA23090073,10A30Q-L.AQ1TL,EKHQ,Y,N,160.0,No Report,NaN,...,NaN,NaN,PC,2023-09-04(W36),KOREA,LGEKR,Korea,2023,09,10A30Q
9,OS_WINGTECH_CN_P,ZPCZ23004575,BAA23090074,10A30Q-L.AQ1TL,EKHQ,Y,N,270.0,No Report,NaN,...,NaN,NaN,PC,2023-09-04(W36),KOREA,LGEKR,Korea,2023,09,10A30Q


In [26]:
sr.pivot_table('Ship', index=['From Site', 'Region', 'Series', 'Mapping Model.Suffix'], columns=['Year', 'Month', 'Week Name'], aggfunc=sum, margins=True).fillna('-')

Year                                                           2022  \
Month                                                            07   
Week Name                                           2022-07-11(W28)   
From Site        Region Series Mapping Model.Suffix                   
OS_WINGTECH_CN_P KOREA  10A30Q 10A30Q-L.AQ12ML                    -   
                               10A30Q-L.AQ14K                  50.0   
                               10A30Q-L.AQ14ML                    -   
                               10A30Q-L.AQ1TL                     -   
                               10A30Q-L.AQ1UL                     -   
                               10A30Q-L.AQ24KN                    -   
                               10A30Q-L.AQ28K                     -   
                               10A30Q-L.AQ28ML                    -   
                               10A30Q-L.AQ48KN                    -   
All                                                            50.0   

Year                                                                 \
Month                                                                 
Week Name                                           2022-07-25(W30)   
From Site        Region Series Mapping Model.Suffix                   
OS_WINGTECH_CN_P KOREA  10A30Q 10A30Q-L.AQ12ML                    -   
                               10A30Q-L.AQ14K                1350.0   
                               10A30Q-L.AQ14ML                    -   
                               10A30Q-L.AQ1TL                     -   
                               10A30Q-L.AQ1UL                     -   
                               10A30Q-L.AQ24KN                 40.0   
                               10A30Q-L.AQ28K                 280.0   
                               10A30Q-L.AQ28ML                    -   
                               10A30Q-L.AQ48KN                 40.0   
All                                                          1710.0   

Year                                                                 \
Month                                                            08   
Week Name                                           2022-08-01(W31)   
From Site        Region Series Mapping Model.Suffix                   
OS_WINGTECH_CN_P KOREA  10A30Q 10A30Q-L.AQ12ML                    -   
                               10A30Q-L.AQ14K                 840.0   
                               10A30Q-L.AQ14ML                 80.0   
                               10A30Q-L.AQ1TL                     -   
                               10A30Q-L.AQ1UL                     -   
                               10A30Q-L.AQ24KN                    -   
                               10A30Q-L.AQ28K                  80.0   
                               10A30Q-L.AQ28ML                 80.0   
                               10A30Q-L.AQ48KN                    -   
All                                                          1080.0   

Year                                                                 \
Month                                                            11   
Week Name                                           2022-11-14(W46)   
From Site        Region Series Mapping Model.Suffix                   
OS_WINGTECH_CN_P KOREA  10A30Q 10A30Q-L.AQ12ML                    -   
                               10A30Q-L.AQ14K                     -   
                               10A30Q-L.AQ14ML                 80.0   
                               10A30Q-L.AQ1TL                     -   
                               10A30Q-L.AQ1UL                     -   
                               10A30Q-L.AQ24KN                    -   
                               10A30Q-L.AQ28K                     -   
                               10A30Q-L.AQ28ML                120.0   
                               10A30Q-L.AQ48KN                    -   
All                                                           200.0   

Year                

# 모니터 2023년 선적 현황

In [3]:
with open('D:/Data/Shipment Plan_Adjusted_new.DB', 'rb') as f:
    spdb = pickle.load(f)
with open('D:/Data/ODM_SR.db', 'rb') as f:
    sr = pickle.load(f)
with open('D:/Data/contact_window.db', 'rb') as f:
    cont = pickle.load(f)
with open('D:/Data/sitemap.db', 'rb') as f:
    site = pickle.load(f)
with open('D:/Data/contact_window.db', 'rb') as f:
    cont = pickle.load(f)

In [4]:
spdb.groupby(['Planweek', 'Category', 'Created_at']).sum('SP').reset_index().iloc[-16:, ] # 선택할 SP의 Version 목록을 출력

,Planweek,Category,Created_at,SP
85,2023-12-11(W50),Latest26w,202312121117,982602.0
86,2023-12-11(W50),Latest26w,202312141259,983191.0
87,2023-12-18(W51),Final26w,202312200913,1048440.0
88,2023-12-18(W51),Final52w,202312191411,1351564.0
89,2023-12-18(W51),Latest26w,202312180928,1035657.0
90,2023-12-18(W51),Latest26w,202312181509,1049245.0
91,2023-12-18(W51),Latest26w,202312190851,1050118.0
92,2023-12-18(W51),Latest26w,202312211119,1047360.0
93,2023-12-25(W52),Final26w,202312271206,1094155.0
94,2023-12-25(W52),Final52w,202312261416,1380956.0


In [15]:
i = 99 # 선택할 SP version 설정

In [16]:
sp = spdb.merge(spdb.groupby(['Planweek', 'Category', 'Created_at']).sum('SP').reset_index().iloc[[i], :3])
sp = pd.merge(sp, site[['To Site', 'Subsidiary', 'Country']], how='left')
sp

,Planweek,Created_at,Division,From Site,Region,To Site,Mapping Model.Suffix,Rep PMS,Category,Frozen,Week Name,SP,Subsidiary,Country
0,2024-01-01(W01),202401021404,MNT,OS_AUO_CN_P,INDIA,EAIL,24GQ50F-B.ATRQ,24GQ50F-BD.CTRVMV,Final52w,N,2024-02-05(W06),572.0,LGEIL,India
1,2024-01-01(W01),202401021404,MNT,OS_AUO_CN_P,INDIA,EAIL,24GQ50F-B.ATRQ,24GQ50F-BD.CTRVMV,Final52w,N,2024-05-13(W20),260.0,LGEIL,India
2,2024-01-01(W01),202401021404,MNT,OS_AUO_CN_P,INDIA,EAIL,27GQ50F-B.ATRQ,27GQ50F-BD.CTRVMV,Final52w,N,2024-03-11(W11),429.0,LGEIL,India
3,2024-01-01(W01),202401021404,MNT,OS_AUO_CN_P,SEA,EAIN_DC,24GQ50F-B.ATIQ,24GQ50F-BD.CTIVMV,Final52w,N,2024-02-19(W08),312.0,LGEIN,Indonesia
4,2024-01-01(W01),202401021404,MNT,OS_AUO_CN_P,SEA,EAIN_DC,24GQ50F-B.ATIQ,24GQ50F-BD.CTIVMV,Final52w,N,2024-03-25(W13),312.0,LGEIN,Indonesia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3500,2024-01-01(W01),202401021404,PC,OS_WINGTECH_CN_P,KOREA,EKHQ,10A30Q-L.AQ1UL,10A30Q-L.AQ1UL,Final52w,Y,2024-01-29(W05),1000.0,LGEKR,Korea
3501,2024-01-01(W01),202401021404,PC,OS_WINGTECH_CN_P,KOREA,EKHQ,10A30Q-L.AQ26ML,10A30Q-L.AQ26ML,Final52w,Y,2024-01-29(W05),52.0,LGEKR,Korea
3502,2024-01-01(W01),202401021404,PC,OS_WINGTECH_CN_P,KOREA,EKHQ,10A30Q-L.AQ28K,10A30Q-L.AQ28K,Final52w,Y,2024-01-08(W02),360.0,LGEKR,Korea
3503,2024-01-01(W01),202401021404,PC,OS_WINGTECH_CN_P,KOREA,EKHQ,10A30Q-L.AQ28K,10A30Q-L.AQ28K,Final52w,Y,2024-01-29(W05),228.0,LGEKR,Korea


In [17]:
sp = sp[sp['From Site'].isin(['OS_TCL_CN_P', 'OS_TPV_CN_P', 'OS_AUO_CN_P'])]
sp['Series'] = sp['Mapping Model.Suffix'].apply(lambda x:x[:2]) # 모니터 모델의 시리즈명(inch) 추출

C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_14776\2792184806.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sp['Series'] = sp['Mapping Model.Suffix'].apply(lambda x:x[:2]) # 모니터 모델의 시리즈명(inch) 추출


In [18]:
sp[['Year', 'Month']] = sp['Week Name'].apply(lambda x:f'{datetime.date.fromisoformat(x[:10]).isocalendar().year}-{get_month_from_date(datetime.date.fromisoformat(x[:10])):02d}').str.split('-', expand=True)

C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_14776\1841757425.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sp[['Year', 'Month']] = sp['Week Name'].apply(lambda x:f'{datetime.date.fromisoformat(x[:10]).isocalendar().year}-{get_month_from_date(datetime.date.fromisoformat(x[:10])):02d}').str.split('-', expand=True)
C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_14776\1841757425.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sp[['Year', 'Month']] = sp['Week Name'].apply(lambda x:f'{datetime.da

In [19]:
c1 = sp['Year'] == '2023'
c2 = sp['Month'] > '11'
sp[c1 & c2]['Week Name'].unique()

array(['2023-12-04(W49)', '2023-12-11(W50)', '2023-12-18(W51)',
       '2023-12-25(W52)'], dtype=object)

In [20]:
sp = sp[sp['Week Name'] >= '2023-12-04(W49)']

In [21]:
sr = sr[sr['Week Name'] < '2023-12-04(W49)']

In [22]:
sr = sr[sr['From Site'].isin(['OS_TCL_CN_P', 'OS_TPV_CN_P', 'OS_AUO_CN_P'])]

In [23]:
sr = sr[['Year', 'Month', 'From Site', 'Division', 'Region', 'Country', 'Subsidiary', 'To Site', 'Series', 'Mapping Model.Suffix', 'Week Name', 'Ship']]

KeyError: "['Series'] not in index"

In [30]:
sr = sr.rename(columns={'Ship':'SP'})

In [31]:
sp

,Planweek,Created_at,Division,From Site,Region,To Site,Mapping Model.Suffix,Rep PMS,Category,Frozen,Week Name,SP,Subsidiary,Country,Series,Year,Month
0,2023-12-18(W51),202312211119,MNT,OS_AUO_CN_P,EU,EEBN,24GQ50F-B.AEUQ,24GQ50F-BD.CEUVMV,Latest26w,N,2024-01-22(W04),312.0,LGEBN,Netherlands,24,2024,01
1,2023-12-18(W51),202312211119,MNT,OS_AUO_CN_P,EU,EEBN,27GQ50F-B.AEUQ,27GQ50F-BD.CEUVMV,Latest26w,N,2024-01-22(W04),195.0,LGEBN,Netherlands,27,2024,01
2,2023-12-18(W51),202312211119,MNT,OS_AUO_CN_P,EU,EEBN,27GQ50F-B.AEUQ,27GQ50F-BD.CEUVMV,Latest26w,N,2024-03-25(W13),429.0,LGEBN,Netherlands,27,2024,03
3,2023-12-18(W51),202312211119,MNT,OS_AUO_CN_P,EU,EEBN,27GQ50F-B.AEUQ,27GQ50F-BD.CEUVMV,Latest26w,N,2024-04-22(W17),429.0,LGEBN,Netherlands,27,2024,04
4,2023-12-18(W51),202312211119,MNT,OS_AUO_CN_P,EU,EEPL_CZ,24GQ50F-B.AEUQ,24GQ50F-BD.CEUVMV,Latest26w,N,2024-03-18(W12),572.0,LGECZ,Czech,24,2024,03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2365,2023-12-18(W51),202312211119,MNT,OS_TPV_CN_P,SEA,EATT,32MR50C-B.ATTQ,32MR50C-BB.CTTVMR,Latest26w,Y,2023-12-18(W51),600.0,LGETT,Taiwan,32,2023,12
2368,2023-12-18(W51),202312211119,MNT,OS_TPV_CN_P,SEA,EAVH_HCM,24MR400-B.ATVQ,24MR400-BB.CTVRMR,Latest26w,Y,2023-12-25(W52),800.0,LGEVH,Vietnam,24,2023,12
2370,2023-12-18(W51),202312211119,MNT,OS_TPV_CN_P,SEA,EAVH_HN,24MR400-B.ATVQ,24MR400-BB.CTVRMR,Latest26w,Y,2023-12-25(W52),800.0,LGEVH,Vietnam,24,2023,12
2371,2023-12-18(W51),202312211119,MNT,OS_TPV_CN_P,SEA,EHAP_LGS,32MR50C-B.AAUQ,32MR50C-BB.CAUVMR,Latest26w,Y,2023-12-25(W52),600.0,LGEAP,Austrailia,32,2023,12


In [35]:
tt = pd.concat([sr, sp], join='inner').reset_index(drop=True)
tt

,Year,Month,From Site,Division,Region,Country,Subsidiary,To Site,Series,Mapping Model.Suffix,Week Name,SP
0,2022,12,OS_AUO_CN_P,MNT,EU,Spain,LGEES,EEES,27,27GQ50F-B.AEUQ,2022-11-28(W48),858.0
1,2022,12,OS_AUO_CN_P,MNT,EU,Spain,LGEES,EEES,24,24GQ50F-B.AEUQ,2022-11-28(W48),1144.0
2,2022,12,OS_AUO_CN_P,MNT,NA,Canada,LGECI,ENCI,24,24GQ50F-B.ACCQ,2022-12-05(W49),572.0
3,2022,12,OS_AUO_CN_P,MNT,SEA,Japan,LGEJP,EJJP,24,24GQ50F-B.AJLQ,2022-12-05(W49),208.0
4,2022,12,OS_AUO_CN_P,MNT,SEA,Japan,LGEJP,EJJP,27,27GQ50F-B.AJLQ,2022-12-05(W49),273.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1669,2023,12,OS_TPV_CN_P,MNT,SEA,Taiwan,LGETT,EATT,32,32MR50C-B.ATTQ,2023-12-18(W51),600.0
1670,2023,12,OS_TPV_CN_P,MNT,SEA,Vietnam,LGEVH,EAVH_HCM,24,24MR400-B.ATVQ,2023-12-25(W52),800.0
1671,2023,12,OS_TPV_CN_P,MNT,SEA,Vietnam,LGEVH,EAVH_HN,24,24MR400-B.ATVQ,2023-12-25(W52),800.0
1672,2023,12,OS_TPV_CN_P,MNT,SEA,Austrailia,LGEAP,EHAP_LGS,32,32MR50C-B.AAUQ,2023-12-25(W52),600.0


In [40]:
tt = tt[tt['Year'] >= '2023']

In [43]:
tt.pivot_table('SP', index=['Series', 'Region'], columns=['Year', 'Month'], aggfunc=sum).fillna('-').to_clipboard()

In [44]:
tt.to_clipboard()

In [6]:
p_wk = spdb.groupby(['Planweek', 'Category', 'Created_at']).sum('SP').reset_index().iloc[i, :3]['Planweek']

In [18]:
wk_list = [get_weekname_from(p_wk, w) for w in range(0, 5)]

In [23]:
c1 = sp['From Site'].str.contains('TCL')
c2 = sp['Week Name'].isin(wk_list)
sp[c1 & c2].sort_values('Week Name')[['Week Name', 'Region', 'Country', 'To Site', 'Mapping Model.Suffix', 'SP']].reset_index(drop=True).to_clipboard()

In [155]:
ref_week = '2023-11-20(W47)'

In [156]:
sp.shape

(2476, 14)

In [157]:
sp = sp[sp['Week Name'] > ref_week]
sp = sp[sp['From Site'].isin(['OS_TCL_CN_P', 'OS_TPV_CN_P', 'OS_AUO_CN_P'])]
sp['Series'] = sp['Mapping Model.Suffix'].apply(lambda x:x[:2]) # 모니터 모델의 시리즈명(inch) 추출
sp = sp[sp['Week Name'] <= '2023-12-25(W52)']
sp = sp.rename(columns={'SP': 'Ship'})


In [158]:
sp

,Planweek,Created_at,Division,From Site,Region,To Site,Mapping Model.Suffix,Rep PMS,Category,Frozen,Week Name,Ship,Subsidiary,Country,Series,Year,Month
1806,2023-12-04(W49),202312060916,MNT,OS_AUO_CN_P,EU,EEBN,24GQ50F-B.AEUQ,24GQ50F-BD.CEUVMV,Final26w,Y,2023-11-27(W48),572.0,LGEBN,Netherlands,24,2023,11
1807,2023-12-04(W49),202312060916,MNT,OS_AUO_CN_P,EU,EEBN,24GQ50F-B.AEUQ,24GQ50F-BD.CEUVMV,Final26w,Y,2023-12-25(W52),312.0,LGEBN,Netherlands,24,2023,12
1808,2023-12-04(W49),202312060916,MNT,OS_AUO_CN_P,EU,EEBN,27GQ50F-B.AEUQ,27GQ50F-BD.CEUVMV,Final26w,Y,2023-12-25(W52),195.0,LGEBN,Netherlands,27,2023,12
1810,2023-12-04(W49),202312060916,MNT,OS_AUO_CN_P,EU,EEDG,27GQ50F-B.AEUQ,27GQ50F-BD.CEUVMV,Final26w,Y,2023-12-04(W49),1080.0,LGEDG,Germany,27,2023,12
1814,2023-12-04(W49),202312060916,MNT,OS_AUO_CN_P,EU,EEES,24GQ50F-B.AEUQ,24GQ50F-BD.CEUVMV,Final26w,Y,2023-12-25(W52),1716.0,LGEES,Spain,24,2023,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2461,2023-12-04(W49),202312060916,MNT,OS_TPV_CN_P,SEA,EAVH_HCM,24MR400-B.ATVQ,24MR400-BB.CTVRMR,Final26w,Y,2023-11-27(W48),800.0,LGEVH,Vietnam,24,2023,11
2462,2023-12-04(W49),202312060916,MNT,OS_TPV_CN_P,SEA,EAVH_HCM,24MR400-B.ATVQ,24MR400-BB.CTVRMR,Final26w,Y,2023-12-25(W52),800.0,LGEVH,Vietnam,24,2023,12
2464,2023-12-04(W49),202312060916,MNT,OS_TPV_CN_P,SEA,EAVH_HN,24MR400-B.ATVQ,24MR400-BB.CTVRMR,Final26w,Y,2023-12-25(W52),800.0,LGEVH,Vietnam,24,2023,12
2465,2023-12-04(W49),202312060916,MNT,OS_TPV_CN_P,SEA,EHAP_LGS,32MR50C-B.AAUQ,32MR50C-BB.CAUVMR,Final26w,Y,2023-12-25(W52),600.0,LGEAP,Austrailia,32,2023,12


In [12]:
sr = sr.reset_index()
sr['Week Name'] = sr['Ship Date'].map(get_weekname)
sr['To Site'] = sr['To Site'].replace(ship_to_map)
sr = pd.merge(sr, site[['Region', 'Subsidiary', 'To Site', 'Country']], how='left')
sr[['Year', 'Month']] = sr['Week Name'].apply(lambda x:f'{datetime.date.fromisoformat(x[:10]).isocalendar().year}-{get_month_from_date(datetime.date.fromisoformat(x[:10])):02d}').str.split('-', expand=True)

c1 = sr['From Site'].str.contains('AUO|TPV|TCL') # 모니터 ODM 일 경우
c2 = sr['Mapping Model.Suffix'].str.match('\d{2}[T|Z|U|A][G|D|N|W|C]?\d{2}[A-Z]{1,2}') # PC/태블릿 모델일 경우
c3 = sr['Mapping Model.Suffix'].str.contains('CL600') # 페가트론 CL600 모델일 경우

sr.loc[c1, 'Series'] = sr[c1]['Mapping Model.Suffix'].apply(lambda x:x[:2]) # 모니터 모델의 시리즈명(inch) 추출
sr.loc[c2, 'Series'] = sr[c2]['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).str.replace('D|G|N|W|C', '').apply(lambda x:x[:6]) # PC모델의 대표모델명 추출
sr.loc[c3, 'Series'] = 'CL600'
sr = sr.sort_values('Ship Date')
sr = sr[['Year', 'Month', 'From Site', 'Division', 'Region', 'Country', 'Subsidiary', 'To Site', 'Series', 'Mapping Model.Suffix', 'PO No.', 'RSD', 'PO1 No', 'Unit Price', 'Method', 'SO No.', 'Ship Date', 'Week Name', 'Ship', 'BL No', 'Final Destination']].reset_index(drop=True)

C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_19524\1839493894.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  sr.loc[c2, 'Series'] = sr[c2]['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).str.replace('D|G|N|W|C', '').apply(lambda x:x[:6]) # PC모델의 대표모델명 추출


In [160]:
sr = sr[sr['From Site'].isin(['OS_TCL_CN_P', 'OS_TPV_CN_P', 'OS_AUO_CN_P'])]
sr = sr[sr['Year'] == '2023']
sr = sr[sr['Week Name'] <= ref_week]

In [161]:
df = pd.concat([sp, sr], join='inner')
df

,Division,From Site,Region,To Site,Mapping Model.Suffix,Week Name,Ship,Subsidiary,Country,Series,Year,Month
1806,MNT,OS_AUO_CN_P,EU,EEBN,24GQ50F-B.AEUQ,2023-11-27(W48),572.0,LGEBN,Netherlands,24,2023,11
1807,MNT,OS_AUO_CN_P,EU,EEBN,24GQ50F-B.AEUQ,2023-12-25(W52),312.0,LGEBN,Netherlands,24,2023,12
1808,MNT,OS_AUO_CN_P,EU,EEBN,27GQ50F-B.AEUQ,2023-12-25(W52),195.0,LGEBN,Netherlands,27,2023,12
1810,MNT,OS_AUO_CN_P,EU,EEDG,27GQ50F-B.AEUQ,2023-12-04(W49),1080.0,LGEDG,Germany,27,2023,12
1814,MNT,OS_AUO_CN_P,EU,EEES,24GQ50F-B.AEUQ,2023-12-25(W52),1716.0,LGEES,Spain,24,2023,12
...,...,...,...,...,...,...,...,...,...,...,...,...
3326,MNT,OS_AUO_CN_P,EU,EERO,27GQ50F-B.AEUQ,2023-11-20(W47),273.0,LGERO,Romania,27,2023,11
3327,MNT,OS_AUO_CN_P,EU,EEMK,24GQ50F-B.AEUQ,2023-11-20(W47),572.0,LGEMK,Hungary,24,2023,11
3328,MNT,OS_TCL_CN_P,EU,EERO,27MR400-B.AEUQ,2023-11-20(W47),256.0,LGERO,Romania,27,2023,11
3329,MNT,OS_AUO_CN_P,EU,EERO,24GQ50F-B.AEUQ,2023-11-20(W47),208.0,LGERO,Romania,24,2023,11


In [166]:
df['SCM'] = df['Subsidiary'].map(cont[(cont['Div']=='MNT') & (cont['Team']=='SCM')].groupby(['Subsidiary'])['Name'].max())
df['MKT'] = df['Subsidiary'].map(cont[(cont['Div']=='MNT') & (cont['Team']=='MKT')].groupby(['Subsidiary'])['Name'].max())
us_b2b = df.loc[df['To Site'].str.contains('ENUS') & (df['Mapping Model.Suffix'].apply(lambda x:x[2]) == 'B'), 'Subsidiary'].map(cont[(cont['Div']=='MNT') & (cont['Team']=='MKT')].groupby(['Subsidiary'])['Name'].min())
df.loc[df['To Site'].str.contains('ENUS') & (df['Mapping Model.Suffix'].apply(lambda x:x[2]) == 'B'), 'MKT'] = us_b2b

In [164]:
df.pivot_table('Ship', index=['From Site', 'Series'], columns=['Year', 'Month'], aggfunc=sum, margins=True).fillna('-').to_clipboard()

In [174]:
df[df['From Site'].str.contains('TCL')].pivot_table('Ship', index=['From Site', 'Region', 'Country', 'Subsidiary', 'To Site', 'MKT', 'Series'], columns=['Year', 'Month'], aggfunc=sum, margins=True).fillna('-').to_clipboard()

In [46]:
df.pivot_table

,Planweek,Created_at,Division,From Site,Region,To Site,Mapping Model.Suffix,Rep PMS,Category,Frozen,Week Name,SP
0,2023-09-04(W36),202309070918,MNT,OS_AUO_CN_P,INDIA,EAIL,24GQ50F-B.ATRQ,24GQ50F-BD.CTRVMV,Final26w,N,2023-12-25(W52),260.0
3,2023-09-04(W36),202309070918,MNT,OS_TCL_CN_P,INDIA,EAIL,22MR410-B.ATRQ,22MR410-BB.CTRRMV,Final26w,N,2023-12-11(W50),840.0
4,2023-09-04(W36),202309070918,MNT,OS_TCL_CN_P,INDIA,EAIL,22MR410-B.ATRQ,22MR410-BB.CTRRMV,Final26w,N,2023-12-25(W52),840.0
8,2023-09-04(W36),202309070918,MNT,OS_TCL_CN_P,INDIA,EAIL,24MR400-B.ATRQ,24MR400-BA.CTRRMV,Final26w,N,2023-11-27(W48),800.0
12,2023-09-04(W36),202309070918,MNT,OS_TCL_CN_P,INDIA,EAIL,27MR400-B.ATRQ,27MR400-BB.CTRRMV,Final26w,N,2023-12-25(W52),512.0
...,...,...,...,...,...,...,...,...,...,...,...,...
186517,2023-12-04(W49),202312060916,MNT,OS_TPV_CN_P,SEA,EAVH_HCM,24MR400-B.ATVQ,24MR400-BB.CTVRMR,Final26w,Y,2023-11-27(W48),800.0
186518,2023-12-04(W49),202312060916,MNT,OS_TPV_CN_P,SEA,EAVH_HCM,24MR400-B.ATVQ,24MR400-BB.CTVRMR,Final26w,Y,2023-12-25(W52),800.0
186520,2023-12-04(W49),202312060916,MNT,OS_TPV_CN_P,SEA,EAVH_HN,24MR400-B.ATVQ,24MR400-BB.CTVRMR,Final26w,Y,2023-12-25(W52),800.0
186521,2023-12-04(W49),202312060916,MNT,OS_TPV_CN_P,SEA,EHAP_LGS,32MR50C-B.AAUQ,32MR50C-BB.CAUVMR,Final26w,Y,2023-12-25(W52),600.0


# 모니터 2024년 사업계획

In [2]:
with open('D:/Data/LG 2024 Monitor Plan.bin', 'rb') as f:
    monitor2024 = pickle.load(f)

In [3]:
monitor2024['Sales'].sum() / monitor2024['Sales Qty'].sum() # 모니터 평균 판가

178.9986439696884

In [10]:
for model in monitor2024['Model']:
    c1 = (monitor2024['Model']==model)
    ttl_amt = monitor2024.loc[c1, 'Sales'].sum()
    ttl_qty = monitor2024.loc[c1, 'Sales Qty'].sum()
    print(f'{model}의 평균 판가는 {ttl_amt/ttl_qty}입니다.')

24GS50F의 평균 판가는 112.60084232375748입니다.
24MR400의 평균 판가는 84.34356654767454입니다.
24MS500의 평균 판가는 80.85072395886932입니다.
27GS85Q의 평균 판가는 275.2880467330413입니다.
27MR400의 평균 판가는 106.89859229762628입니다.
27SR50F의 평균 판가는 146.3628791303994입니다.
27US750의 평균 판가는 220.7424402509695입니다.
29WQ600의 평균 판가는 147.1004801058242입니다.
34WR55QC의 평균 판가는 217.8697850120007입니다.
VM의 평균 판가는 inf입니다.
VM의 평균 판가는 inf입니다.
22MR410의 평균 판가는 75.2527545278743입니다.
24GS50F의 평균 판가는 112.60084232375748입니다.
24MR41S의 평균 판가는 64.4569215060543입니다.
24MS500의 평균 판가는 80.85072395886932입니다.
27GS50F의 평균 판가는 127.53632639958077입니다.
27SR73U의 평균 판가는 216.56650809617292입니다.
32SR70U의 평균 판가는 292.64232727051206입니다.
34WR55QC의 평균 판가는 217.8697850120007입니다.
VM의 평균 판가는 inf입니다.
VM의 평균 판가는 inf입니다.
22MS31W의 평균 판가는 49.85468114832803입니다.
24GS60F의 평균 판가는 119.68289475468718입니다.
24MS500의 평균 판가는 80.85072395886932입니다.
27GS60F의 평균 판가는 154.17069891614153입니다.
27GS85Q의 평균 판가는 275.2880467330413입니다.
27MS500의 평균 판가는 104.33336771328074입니다.
27SR75U의 평균 판가는 204.90534673740171입니다.
27

C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_2860\380030066.py:5: RuntimeWarning: divide by zero encountered in double_scalars
  print(f'{model}의 평균 판가는 {ttl_amt/ttl_qty}입니다.')


27MS550의 평균 판가는 94.09488670573548입니다.
27SR50F의 평균 판가는 146.3628791303994입니다.
27UQ850V의 평균 판가는 291.2116594054453입니다.
27UQ85RV의 평균 판가는 267.66842109642874입니다.
32GS85Q의 평균 판가는 277.14709871348066입니다.
32SR50F의 평균 판가는 164.2220802916114입니다.
32SR83U의 평균 판가는 327.78796256497003입니다.
32UQ850V의 평균 판가는 479.7732774366307입니다.
32UQ85RV의 평균 판가는 387.31858824993566입니다.
32UR500의 평균 판가는 228.59706523639363입니다.
34GN850의 평균 판가는 428.9629290863368입니다.
34WR50QC의 평균 판가는 178.67277362162386입니다.
38WR85QC의 평균 판가는 536.9566166030269입니다.
43SQ700S의 평균 판가는 318.7863958840848입니다.
45GR75DC의 평균 판가는 566.3243257040501입니다.
32HL512D의 평균 판가는 1628.4429501980223입니다.
27HJ713C의 평균 판가는 569.8021035778501입니다.
32HL714S의 평균 판가는 2987.1973642656344입니다.
27GS95QE의 평균 판가는 490.20969285993795입니다.
32GS95UE의 평균 판가는 585.5645799336081입니다.
34GS95QE의 평균 판가는 623.7566932141812입니다.
39GS95QE의 평균 판가는 689.8651804574279입니다.
45GS95QE의 평균 판가는 773.2879756233201입니다.
VM의 평균 판가는 inf입니다.
VM의 평균 판가는 inf입니다.
24MS500의 평균 판가는 80.85072395886932입니다.
25MS500의 평균 판가는 69.623596

34WR55QC의 평균 판가는 217.8697850120007입니다.
38WR85QC의 평균 판가는 536.9566166030269입니다.
VM의 평균 판가는 inf입니다.
VM의 평균 판가는 inf입니다.
24CN650N의 평균 판가는 347.3227619763904입니다.
24TQ510S의 평균 판가는 116.10456278749972입니다.
24BA550의 평균 판가는 83.61710906638754입니다.
24GS50F의 평균 판가는 112.60084232375748입니다.
24MR400의 평균 판가는 84.34356654767454입니다.
24MS500의 평균 판가는 80.85072395886932입니다.
27TQ615S의 평균 판가는 136.01581991803286입니다.
27BA550의 평균 판가는 108.74928539485367입니다.
27GS50F의 평균 판가는 127.53632639958077입니다.
27GS60QC의 평균 판가는 119.96892543815649입니다.
27MS500의 평균 판가는 104.33336771328074입니다.
27US500의 평균 판가는 220.67327722282798입니다.
32GS75Q의 평균 판가는 231.1520106633985입니다.
32MR50C의 평균 판가는 119.3405846856461입니다.
34WR55QC의 평균 판가는 217.8697850120007입니다.
38WR85QC의 평균 판가는 536.9566166030269입니다.
27GS95QE의 평균 판가는 490.20969285993795입니다.
32GS95UE의 평균 판가는 585.5645799336081입니다.
34GS95QE의 평균 판가는 623.7566932141812입니다.
39GS95QE의 평균 판가는 689.8651804574279입니다.
45GS95QE의 평균 판가는 773.2879756233201입니다.
VM의 평균 판가는 inf입니다.
VM의 평균 판가는 inf입니다.
VM의 평균 판가는 inf입니다.
24TQ510S의

24MR400의 평균 판가는 84.34356654767454입니다.
24MS500의 평균 판가는 80.85072395886932입니다.
27TQ615S의 평균 판가는 136.01581991803286입니다.
27BA550의 평균 판가는 108.74928539485367입니다.
27GS50F의 평균 판가는 127.53632639958077입니다.
27GS60QC의 평균 판가는 119.96892543815649입니다.
27MS500의 평균 판가는 104.33336771328074입니다.
27SR50F의 평균 판가는 146.3628791303994입니다.
27US500의 평균 판가는 220.67327722282798입니다.
32GS75Q의 평균 판가는 231.1520106633985입니다.
32MR50C의 평균 판가는 119.3405846856461입니다.
32SQ700S의 평균 판가는 222.69373272999619입니다.
32SR50F의 평균 판가는 164.2220802916114입니다.
34WR55QC의 평균 판가는 217.8697850120007입니다.
38WR85QC의 평균 판가는 536.9566166030269입니다.
21HQ613D의 평균 판가는 1872.827588227554입니다.
14HQ701G의 평균 판가는 5895.022808238113입니다.
24HR513C의 평균 판가는 708.5368643137263입니다.
32HR734S의 평균 판가는 2292.404065396311입니다.
27GS95QE의 평균 판가는 490.20969285993795입니다.
32GS95UE의 평균 판가는 585.5645799336081입니다.
34GS95QE의 평균 판가는 623.7566932141812입니다.
39GS95QE의 평균 판가는 689.8651804574279입니다.
45GS95QE의 평균 판가는 773.2879756233201입니다.
VM의 평균 판가는 inf입니다.
VM의 평균 판가는 inf입니다.
VM의 평균 판가는 inf입니다.
24CN650N의

...

27MR41S의 평균 판가는 81.13043149136873입니다.
27MS550의 평균 판가는 94.09488670573548입니다.
27QN600의 평균 판가는 173.13439264535754입니다.
27SR50F의 평균 판가는 146.3628791303994입니다.
27UL500의 평균 판가는 205.79012862727802입니다.
28MQ780의 평균 판가는 489.4424772616694입니다.
29WL500의 평균 판가는 135.22736133773154입니다.
32SP510M의 평균 판가는 147.8835308899093입니다.
32GN600의 평균 판가는 156.4784927332331입니다.
32GN650의 평균 판가는 196.17310203142725입니다.
32MP60G의 평균 판가는 126.29781517224693입니다.
32QN600의 평균 판가는 188.49449872538602입니다.
32QN650의 평균 판가는 212.00211289041007입니다.
32SR50F의 평균 판가는 164.2220802916114입니다.
32UP550N의 평균 판가는 237.02105810374354입니다.
35WN75CN의 평균 판가는 297.5495125868589입니다.
43SQ700S의 평균 판가는 318.7863958840848입니다.
49GR85DC의 평균 판가는 640.8467772309359입니다.
49WQ95C의 평균 판가는 765.0099123097046입니다.
21HQ513D의 평균 판가는 1604.6685077692487입니다.
10HQ701G의 평균 판가는 6035.804780249962입니다.
14HQ701G의 평균 판가는 5895.022808238113입니다.
14HQ901G의 평균 판가는 6813.365262405254입니다.
24HR513C의 평균 판가는 708.5368643137263입니다.
27HK510S의 평균 판가는 1115.2144342896656입니다.
32HL710S의 평균 판가는 2278.2036582

34SS60QC의 평균 판가는 211.35854669498843입니다.
34SS65QC의 평균 판가는 224.26441851895135입니다.
34WR55QC의 평균 판가는 217.8697850120007입니다.
37US750의 평균 판가는 328.9015699614214입니다.
38WR85QC의 평균 판가는 536.9566166030269입니다.
21HQ613D의 평균 판가는 1872.827588227554입니다.
14HQ701G의 평균 판가는 5895.022808238113입니다.
24HR513C의 평균 판가는 708.5368643137263입니다.
32HR734S의 평균 판가는 2292.404065396311입니다.
27GS93QE의 평균 판가는 509.79486806907823입니다.
27GS95QE의 평균 판가는 490.20969285993795입니다.
32GS95UE의 평균 판가는 585.5645799336081입니다.
34GS95QE의 평균 판가는 623.7566932141812입니다.
39GS95QE의 평균 판가는 689.8651804574279입니다.
45GS95QE의 평균 판가는 773.2879756233201입니다.
45GS96QB의 평균 판가는 890.679033914292입니다.
VM의 평균 판가는 inf입니다.
VM의 평균 판가는 inf입니다.
22MP410의 평균 판가는 66.27014706864264입니다.
22MR41S의 평균 판가는 57.94366690539477입니다.
22MS31W의 평균 판가는 49.85468114832803입니다.
24BA450의 평균 판가는 125.67508048132017입니다.
24BA550의 평균 판가는 83.61710906638754입니다.
24BA560의 평균 판가는 157.17007005697266입니다.
24GS50F의 평균 판가는 112.60084232375748입니다.
24GS60F의 평균 판가는 119.68289475468718입니다.
24MC400의 평균 판가는 70.757859269

24GQ50F의 평균 판가는 132.49616801790592입니다.
24BK430H의 평균 판가는 83.5643506196243입니다.
24BK550Y의 평균 판가는 121.81990433875869입니다.
24BL650C의 평균 판가는 145.86406237693322입니다.
24GS50F의 평균 판가는 112.60084232375748입니다.
24GS60F의 평균 판가는 119.68289475468718입니다.
24MP400의 평균 판가는 78.4560228139916입니다.
24MP60G의 평균 판가는 84.15538317127672입니다.
24MR400의 평균 판가는 84.34356654767454입니다.
24MS500의 평균 판가는 80.85072395886932입니다.
24MS550의 평균 판가는 75.09710512608517입니다.
24MS55C의 평균 판가는 83.91644714191578입니다.
24QP500의 평균 판가는 132.7100348111295입니다.
25MS500의 평균 판가는 69.6235965875596입니다.
25MS550의 평균 판가는 71.98343614234413입니다.
25SR50F의 평균 판가는 113.36202410503292입니다.
27LQ625S의 평균 판가는 130.92656649148302입니다.
27GQ40W의 평균 판가는 93.57033331109437입니다.
27BK430H의 평균 판가는 114.92088839407207입니다.
27BK550Y의 평균 판가는 137.33186055371945입니다.
27BK67U의 평균 판가는 308.06076732132277입니다.
27BL650C의 평균 판가는 166.4470901699782입니다.
27GL850의 평균 판가는 198.73487806387632입니다.
27GN800의 평균 판가는 168.7096668130639입니다.
27GP700의 평균 판가는 159.6400762745332입니다.
27GP83B의 평균 판가는 218.85231093556183입

32SQ780S의 평균 판가는 316.45797491062996입니다.
32SR53FS의 평균 판가는 157.0669378693128입니다.
32SR70U의 평균 판가는 292.64232727051206입니다.
32SR73U의 평균 판가는 225.55241777054124입니다.
32SR75U의 평균 판가는 241.3641607013604입니다.
32UL500의 평균 판가는 201.62131886689477입니다.
32UN500의 평균 판가는 208.01658968586258입니다.
32UN550의 평균 판가는 189.99326478840905입니다.
32UN650의 평균 판가는 278.3624824819914입니다.
32UN880의 평균 판가는 415.54100461456585입니다.
32UP550N의 평균 판가는 237.02105810374354입니다.
32UQ750의 평균 판가는 400.41076612567076입니다.
34BK95U의 평균 판가는 846.9998677689946입니다.
34BL650의 평균 판가는 253.424입니다.
34BN670의 평균 판가는 228.76868275988554입니다.
34BN770의 평균 판가는 296.0526443013195입니다.
34BP65C의 평균 판가는 292.8864192081866입니다.
34BP85CN의 평균 판가는 490.78309617532847입니다.
34BP88CN의 평균 판가는 514.5329199996371입니다.
34BQ60QC의 평균 판가는 292.88786475772577입니다.
34BQ650의 평균 판가는 220.85324272971337입니다.
34BQ680의 평균 판가는 284.970516734765입니다.
34BQ77QB의 평균 판가는 275.467571274816입니다.
34BQ77QC의 평균 판가는 403.70986118631356입니다.
34BR55QC의 평균 판가는 253.30406040315268입니다.
34BR65F의 평균 판가는 223.0355620685415입니다.


32UR500의 평균 판가는 228.59706523639363입니다.
34WP500의 평균 판가는 197.8008019823086입니다.
34WR50QC의 평균 판가는 178.67277362162386입니다.
VM의 평균 판가는 inf입니다.
22MP410의 평균 판가는 66.27014706864264입니다.
24GN60R의 평균 판가는 145.2159291592698입니다.
24MP400의 평균 판가는 78.4560228139916입니다.
24MP60G의 평균 판가는 84.15538317127672입니다.
26WQ500의 평균 판가는 113.11144569047293입니다.
27MR400의 평균 판가는 106.89859229762628입니다.
27GN60R의 평균 판가는 146.3288538687643입니다.
27GP750의 평균 판가는 225.8117442739375입니다.
27GR93U의 평균 판가는 422.0533525690242입니다.
27QN600의 평균 판가는 173.13439264535754입니다.
27UL500의 평균 판가는 205.79012862727802입니다.
27UP650의 평균 판가는 234.6690100784234입니다.
27UP850N의 평균 판가는 241.93770360083533입니다.
29WP500의 평균 판가는 161.43205676039918입니다.
29WQ600의 평균 판가는 147.1004801058242입니다.
32MP60G의 평균 판가는 126.29781517224693입니다.
32QN600의 평균 판가는 188.49449872538602입니다.
34WP500의 평균 판가는 197.8008019823086입니다.
34WR50QC의 평균 판가는 178.67277362162386입니다.
VM의 평균 판가는 inf입니다.
VM의 평균 판가는 inf입니다.
VM의 평균 판가는 inf입니다.
VM의 평균 판가는 inf입니다.
VM의 평균 판가는 inf입니다.
VM의 평균 판가는 inf입니다.
VM의 평균 판가는 inf입니다.

In [202]:
monitor2024['Sales'].sum() # 모니터 총 금액
monitor2024['Sales Qty'].sum() # 모니터 총 수량
monitor2024['Subsidiary(R&R)'].unique() # 법인 현황
monitor2024['Region(Subsidiary)'].unique() # 지역 현황
monitor2024['Country'].unique() # 국가현황
monitor2024['제품군'].unique() # 제품군 현황

array(['Gaming', '일반 Monitor', 'UHD', 'AR 21:9', 'Others',
       'Medical monitor', 'Medical Monitor', 'TV Monitor', 'Cloud'],
      dtype=object)

In [198]:
monitor2024.pivot_table('Sales', index=['Region(R&R)'], columns=['YYYYMM'], aggfunc=sum, margins=True).astype(int) # 모니터 지역별 금액

YYYYMM,202401,202402,202403,202404,202405,202406,202407,202408,202409,202410,202411,202412,All
Region(R&R),,,,,,,,,,,,,
Asia,21081482,20649960,21734491,21858035,24542866,23007517,20511281,21087741,20650230,23709319,26113154,19797787,264743868
Europe,30352153,25708500,28316261,28941530,31341691,30619471,29634000,29281652,35877238,40398852,44613406,31975705,387060464
India,4054581,3283220,4634826,4511186,4422664,4156518,3487858,4441081,5827835,4920603,5054873,4181299,52976551
LATAM,12622317,10924388,14621749,14113763,14428677,14452341,14552990,14717923,14834664,14981548,15683276,12221778,168155420
ME & Africa,4604862,4062514,5478002,4438177,5151832,4844436,4554667,6191397,6755709,5394344,5573319,4326139,61375402
North America,48583164,47768276,51632358,47807143,48411510,61829305,58171446,62177571,59971137,82430713,83088482,58214153,710085264
OTHER,1541434,1547315,1547315,1910075,1910663,1908311,2330583,2329995,2330583,2874608,2875784,2874020,25980691
Parent,21664085,18342297,17746781,13701618,14226381,13035854,13724560,13280742,14559489,13500957,14459196,11070248,179312214
Russia,682005,585755,849779,600346,603541,755899,705721,804896,1136434,1215812,1273246,712698,9926139


In [203]:
monitor2024.pivot_table('Sales', index=['Model'], columns=['YYYYMM'], aggfunc=sum, margins=True).astype(int) # 모니터 모델별 금액

YYYYMM,202401,202402,202403,202404,202405,202406,202407,202408,202409,202410,202411,202412,All
Model,,,,,,,,,,,,,
10HQ701G,0,291523,291523,0,8100,0,0,291882,397629,100679,108779,67119,1557237
14HQ701G,517051,826799,942059,926444,907891,1274436,840188,1092732,1253582,1234907,1604762,1406713,12827569
14HQ901G,0,0,0,34080,34080,34080,34038,34038,34038,34080,34080,34080,306601
17BR30T,17000,17000,17000,17000,17000,17000,20400,20400,20400,20400,20400,20400,224400
17HQ701G,425490,289828,471831,353266,444089,309393,524480,313960,619983,398096,587748,405681,5143851
...,...,...,...,...,...,...,...,...,...,...,...,...,...
CQ601I,57912,57912,86868,86859,57906,72382,57267,57267,57267,73805,73805,73805,813057
CQ601N,27471,27471,41206,41203,27468,41203,27165,27165,27165,42013,42013,42013,413557
CQ601W,10023,10023,10023,10022,10022,10022,9911,9911,9911,10219,10219,10219,120529


In [216]:
go.Figure(data=go.Bar(x=monitor2024.groupby('Model').sum('Sales')['Sales'].index, y=monitor2024.groupby('Model').sum('Sales')['Sales'])) # 모델별 금액

In [204]:
monitor2024.pivot_table('Sales', index=['제품군'], columns=['YYYYMM'], aggfunc=sum, margins=True).astype(int) # 모니터 제품군별 금액

YYYYMM,202401,202402,202403,202404,202405,202406,202407,202408,202409,202410,202411,202412,All
제품군,,,,,,,,,,,,,
AR 21:9,18854475,17574858,19765780,18730101,19959987,21482388,19817832,21224939,22037950,26125816,26523458,19215663,251313252
Cloud,1653996,1368360,1649521,1831238,1459421,1712507,2022631,1861565,2024943,2478115,2204999,2142866,22410166
Gaming,52942491,47625013,50972908,49860191,51588105,55373881,52096775,55164163,55226026,68684054,72047487,52397012,663978111
Medical Monitor,305440,472881,632988,513846,698204,567920,605227,648948,687512,581135,662901,601070,6978077
Medical monitor,2737874,3508548,4388202,4004548,4453971,4794427,4432813,4985409,6477134,5886778,6983084,6053702,58706497
Others,2056592,2030261,2027414,2075400,2081524,2070294,2206131,2203338,2217260,2522113,2533782,2484582,26508698
TV Monitor,4608944,3846411,3975845,3807014,4460122,4700750,4424278,4264062,5071393,5214664,5871836,4301274,54546600
UHD,19715094,18072030,19626941,19142004,19126123,21096561,19579447,20371862,22571969,26514228,28107915,20224391,254148570
일반 Monitor,42311178,38373863,43521963,37917530,41212366,42810923,42487974,43588713,45629132,51419854,53799274,37953266,521026041


In [213]:
fig = go.Figure(data=go.Bar(x=monitor2024.groupby('제품군').sum('Sales')['Sales'].index, y=monitor2024.groupby('제품군').sum('Sales')['Sales']))
fig.update_layout(title='2024년 모니터 제품군별 금액($)', xaxis_title='제품군', yaxis_title='금액($)')

In [195]:
with open('D:/Data/LG 2024 Monitor Plan.bin', 'wb') as f: # 저장
    pickle.dump(monitor_2024, f)

In [25]:
sr.pivot_table('Ship', index=['From Site', 'Series',], columns=['Year', 'Month'], aggfunc=sum, margins=True).fillna('-').to_clipboard()

In [4]:
sr.loc['OS_TCL_CN_P'].pivot_table('Ship', )

,,Mapping Model.Suffix,To Site,Shipping,Cancel,PO,OQC Report,OQC Date,OQC Result,Ship,Issued Date,...,BL Status,Method,Price Term,Unit Price,Currency,Payment Term,PO1 No,SO No.,Final Destination,Division
PO No.,BL No,,,,,,,,,,,,,,,,,,,,,
JGMZ23019574,PLISZ4E08668,22MR410-B.AEUQ,EEMK,Y,N,840.0,Completed,2023-08-16 00:00:00.0,Approved,840.0,2023-07-14 14:42:39,...,CC Requested,SEA,FOB,44.79,USD,Z0090FB,1071964,380166068,HUSZ2,MNT
JGMZ23020571,PLISZ4E09317,22MR410-B.AEUQ,EEES,Y,N,838.0,Completed,2023-08-20 00:00:00.0,Approved,838.0,2023-07-25 14:27:25,...,CC Requested,SEA,FOB,44.79,USD,Z0090FB,1183934,380166821,ESGU2,MNT
JGMZ23022347,PLISZ4E09307,22MR410-B.APDQ,EMTK,Y,N,840.0,Completed,2023-08-20 00:00:00.0,Approved,840.0,2023-08-15 14:50:11,...,CC Requested,SEA,FOB,44.55,USD,Z0090FB,1337544,380166913,TRTUZ,MNT
JGMZ23019775,PLISZ4E09225,22MR410-B.ATMQ,EATH,Y,N,2184.0,Completed,2023-08-20 00:00:00.0,Approved,2184.0,2023-07-18 14:10:07,...,CC Requested,SEA,FOB,44.16,USD,Z0090FB,496087,380166489,THRYG,MNT
JGMZ23020572,PLISZ4E09225,22MR410-B.ATMQ,EATH,Y,N,2184.0,Completed,2023-08-23 00:00:00.0,Approved,2184.0,2023-07-25 14:27:26,...,CC Requested,SEA,FOB,44.16,USD,Z0090FB,496200,380166822,THRYG,MNT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
JGMZ23032705,PLISZ4E13445,27MR400-B.AUSQ,ENCI,Y,N,1280.0,Completed,2023-11-18 00:00:00.0,Approved,1280.0,2023-11-13 21:24:05,...,SD Confirmed,SEA,FOB,57.65,USD,Z0090FB,1080023,380170695,CABUB,MNT
JGMZ23031330,PLISZ4E13547,27MR400-B.AUSQ,ENUS_CA,Y,N,1280.0,Completed,2023-11-19 00:00:00.0,Approved,1280.0,2023-11-01 03:22:49,...,SD Confirmed,SEA,FOB,57.65,USD,Z0090FB,1819566,380170989,USSBT,MNT
ZGMZ23029599,PLISZ4E13556,27MR400.AKRQ,EKHQ,Y,N,1280.0,Completed,2023-11-11 00:00:00.0,Approved,1280.0,2023-10-17 13:53:49,...,SD Confirmed,SEA,FOB,57.81,USD,Z0090FB,NaN,NaN,NaN,MNT
